# Keywords Clustering

In [15]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.cluster import KMeans, k_means_, DBSCAN, AgglomerativeClustering
from sklearn import cluster
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_distances

import hdbscan
from gensim import models
from gensim.corpora import mmcorpus, Dictionary
from gensim.models import lsimodel, ldamodel, tfidfmodel, rpmodel, logentropy_model, TfidfModel, LsiModel, LdaModel
from gensim import matutils, corpora
from gensim.models.doc2vec import TaggedDocument
from gensim.models import doc2vec
from gensim.models.word2vec import Word2Vec
from sklearn.svm import SVC
import sys
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import html
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import chat_bot as cb
import Database_Handler as dh
from datetime import datetime
import seaborn as sns
import functools
from konlpy.tag import Mecab
from konlpy.utils import pprint
mecab = Mecab()
%matplotlib inline
from tqdm import tqdm

## Naver

In [3]:
dataDict = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled','rb'))
print (len(dataDict))

keywordsDict = pickle.load(open('./data/pre_data/keywords/keywords_Naver.pickled','rb'))

for idx in dataDict:
    dataDict[idx]['extracted_keywords'] = keywordsDict[idx]

df = pd.DataFrame.from_dict(dataDict, orient='index')
df['date'] = pd.to_datetime(df['date']).dt.date
df.reset_index(inplace = True)
df.rename(columns={'index':'id'}, inplace=True)
print (df.shape)
df.head()

7681
(7681, 11)


,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a29c445588c132954d1973a,정치,2017-12-07,연합뉴스,1713,1474,1,"北외무성 ""전쟁 바라지 않지만 결코 피하지 않을 것""","美고위인사 대북언급 비난하며 ""전쟁 기정사실화"" 위협 며칠 새 이어지는 북한 군민...","[외무성, 핵전쟁, 대변인]","{중앙, 핵전쟁, 북한, 조선반도, 고위, 대변인, 도화선, 미국}"
1,5a29c445588c132954d1973b,정치,2017-12-07,한국일보,2551,2121,2,"예산전쟁, 예결위 간사ㆍ호남이 웃었다",예결위 간사들이 최대 수혜자..당 지도부 내 몫 챙기기도 여전 황주홍ㆍ김도읍 등...,"[예산, 예결위, soc]","{지역구, 예산안, 정부안, 호남, 의원, 국민의당, 증액}"
2,5a29c445588c132954d1973c,정치,2017-12-07,뉴시스,610,540,3,"혐의 부인에 20시간 조사…檢, 최경환 구속 카드 꺼내나",【서울=뉴시스】 최진석 기자 = 박근혜 정부 시절 국가정보원 특수활동비 수수 의혹 ...,"[최경환, 구속영장, 국가정보원]","{정기국회, 국정원장, 혐의, 의원, 검찰, 조사, 구속영장 청구}"
3,5a29c445588c132954d1973d,정치,2017-12-07,연합뉴스,145,134,4,"최재형 감사원장 후보자 ""독립성 강화는 임명권자의 뜻""",감사원장에 내정된 최재형 사법연수원장(고양=연합뉴스) 이희열 기자 = 7일 감사원장...,"[이슈 · 최재형 감사원장 내정, 감사원장, 최재형, 감사원]","{공직 사회, 법관, 생활, 지명, 후보자, 감사원장}"
4,5a29c445588c132954d1973e,정치,2017-12-07,동아일보,1074,937,5,"B-1B 한반도에 뜨자, 평양 비운 김정은",[동아일보] 북중 접경지 양강도 삼지연 시찰… 방북 유엔 사무차장 면담 안할듯 B-...,"[김정은, b-1b, 한반도]","{펠트먼, 공장, 시찰, 삼지연, 접경, 양강도, 사무차장, 훈련, 김정은}"


## Daum

In [4]:
dataDict2 = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
print (len(dataDict2))

keywordsDict2 = pickle.load(open('./data/pre_data/keywords/keywords_daum.pickled','rb'))

for idx in dataDict2:
    dataDict2[idx]['extracted_keywords'] = keywordsDict2[idx]

df2 = pd.DataFrame.from_dict(dataDict2, orient='index')
df2['date'] = pd.to_datetime(df2['date']).dt.date
df2.reset_index(inplace = True)
df2.rename(columns={'index':'id'}, inplace=True)
print (df2.shape)
df2.head()

4722
(4722, 11)


,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a2a61bf588c13481c229d1e,뉴스,2017-12-07,세계일보,1093,911,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","[부작용, 비아그라, 아일랜드]","{남성들, 공장, 부작용, 지역, 세보 효과, 건강, 연기}"
1,5a2a61bf588c13481c229d1f,뉴스,2017-12-07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"[커피, 낮잠, 음식물]","{치아, 점심 식사, 디스크, 낮잠, 자세, 건강, 식후, 커피, 입냄새, 철분}"
2,5a2a61bf588c13481c229d20,뉴스,2017-12-07,연합뉴스,1067,811,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"[살인혐의, 철원, 검찰송치]","{성관계, 진술, 철원, 아들, 경찰, 범행, 지인, 주변, 남편, 앙심}"
3,5a2a61bf588c13481c229d21,뉴스,2017-12-07,헤럴드경제,418,369,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","[신영자, 갑질, 롯데백화점]","{네이처리퍼블릭, 유통업체, 혐의, 매장, 롯데, 검찰, 신영자 이사장, 징역}"
4,5a2a61bf588c13481c229d22,뉴스,2017-12-07,연합뉴스,434,368,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"[공범, 살인, 과자]","{폭행, 경찰, 범행, 혐의, 쪽지, 과자, 유치장, 남자친구}"


In [5]:
def nav_tokenizer(corpus):
    pos = mecab.pos(corpus)
    res = [x[0] for x in pos if (x[1] == u'VV' or x[1] == u'VA' or x[1] == u'NNB' or x[1] == u'NNP' or x[1] == u'NNG')]
    return res

In [6]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))
def remove_stopwords(words, stopwords):
    res = [x for x in words if x not in stopwords]
    return res

In [13]:
corpus = np.concatenate((df.title.values+'.\n'+df.mainText.values,
                        df2.title.values+'.\n'+df2.mainText.values))
len(corpus)

12403

In [17]:
vect0 = CountVectorizer(tokenizer=nav_tokenizer).fit(tqdm(corpus))
temp = pd.DataFrame(list(vect0.vocabulary_.keys()), columns=['word'])
temp['idx'] = vect0.vocabulary_.values()
temp.sort_values(by='idx', inplace=True)
temp.drop('idx', axis=1, inplace=True)
temp['count'] = vect0.transform(corpus).toarray().sum(axis=0)
temp.sort_values(['count'], ascending=False).head(10)

,word,count
5,것,44316
226,있,40526
116,수,26881
32,등,24590
88,하,22918
43,기자,19123
117,없,17961
73,말,16773
453,받,15411
194,중,11396


In [18]:
vect = TfidfVectorizer(tokenizer=nav_tokenizer, stop_words=stopwords)
vect.fit(tqdm(corpus))
print("token n/a/v: %d" % len(vect.vocabulary_))

100%|██████████| 12403/12403 [00:58<00:00, 213.32it/s]


token n/a/v: 80715


In [19]:
mask1 = [x for x in vect.vocabulary_ if len(x) == 1]
len(mask1)

1695

In [20]:
vect = TfidfVectorizer(tokenizer=nav_tokenizer, stop_words=stopwords + mask1)
vect.fit(tqdm(corpus))
print("token n/a/v: %d" % len(vect.vocabulary_))

100%|██████████| 12403/12403 [00:56<00:00, 217.63it/s]


token n/a/v: 79020


In [21]:
X = vect.transform(corpus).toarray()

In [23]:
lda_docs = [nav_tokenizer(x) for x in tqdm(corpus)]
lda_docs = [remove_stopwords(x, stopwords + mask1) for x in tqdm(lda_docs)]
lda_dict = corpora.Dictionary(lda_docs)

100%|██████████| 12403/12403 [03:39<00:00, 56.50it/s]


In [24]:
tf_ps = [lda_dict.doc2bow(x) for x in tqdm(lda_docs)]
tf_model = models.TfidfModel(tf_ps)
X_lda = tf_model[tf_ps]
corpora.MmCorpus.serialize('ps.mm', X_lda)

100%|██████████| 12403/12403 [00:02<00:00, 4947.30it/s]


In [25]:
# train topic model
ntopics =10
nwords = 5

In [26]:
# LDA
np.random.seed(1)
lda = models.ldamodel.LdaModel(X_lda, id2word=lda_dict, num_topics=ntopics)
pprint(lda.print_topics(num_topics=ntopics, num_words=nwords))

[(0, '0.002*"김현수" + 0.002*"양현종" + 0.002*"경찰" + 0.002*"이상민" + 0.002*"데얀"'),
 (1, '0.002*"대통령" + 0.002*"일본" + 0.002*"정소민" + 0.002*"북한" + 0.002*"중국"'),
 (2, '0.006*"전현무" + 0.002*"추자현" + 0.002*"송중기" + 0.002*"한혜진" + 0.002*"송혜교"'),
 (3, '0.004*"연예대상" + 0.003*"집사부일체" + 0.003*"강호동" + 0.002*"이상윤" + 0.002*"강식당"'),
 (4, '0.007*"선수" + 0.005*"손흥민" + 0.005*"경기" + 0.004*"감독" + 0.004*"계약"'),
 (5, '0.004*"대상" + 0.003*"유재석" + 0.003*"수상" + 0.003*"지성" + 0.003*"연기"'),
 (6, '0.005*"이준" + 0.003*"백종원" + 0.002*"일레븐" + 0.001*"팔라우" + 0.001*"번리"'),
 (7, '0.003*"고준희" + 0.003*"사이보그" + 0.003*"천호진" + 0.003*"이승기" + 0.002*"화유기"'),
 (8, '0.001*"류진" + 0.001*"이강인" + 0.001*"한양" + 0.001*"발렌시아" + 0.001*"독대"'),
 (9, '0.003*"박나래" + 0.003*"연기" + 0.003*"드라마" + 0.003*"이주연" + 0.002*"방송"')]


In [33]:
# get labem from lda model
def get_labels(model, tres, sentence):
    lda = model
    proba_list = lda[sentence]
    res = [x for x in proba_list if x[1] >= tres]
    if len(res) == 0:
        res = None
    return res

In [34]:
dfRes = pd.DataFrame(corpus, columns=['text'])
for i in range(10):
    dfRes[i] = 0; 
dfRes['unknown'] = 0
    
for i in range(len(tf_ps)):
    temp = get_labels(lda, 0.3, tf_ps[i])
    if temp != None:
        idx = [x[0] for x in temp]
        dfRes.set_value(i, idx, 1)
    elif temp == None:
        dfRes.set_value(i, ['unknown'], 1)

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':


In [35]:
# number of reviews for each label
for i in range(10):
    print('label({}): {}'.format(i, len(dfRes[(dfRes[i] == 1)])))
print('unknown: {}'.format(len(dfRes[(dfRes['unknown'] == 1)])))

label(0): 3881
label(1): 5429
label(2): 38
label(3): 71
label(4): 2727
label(5): 282
label(6): 19
label(7): 114
label(8): 10
label(9): 3245
unknown: 106


In [36]:
d2v_docs = [nav_tokenizer(x) for x in corpus]
d2v_docs = [remove_stopwords(x, (stopwords + mask1)) for x in d2v_docs]
tagged_d2v_docs = [TaggedDocument(doc, [idx]) for idx, doc in tqdm(enumerate(d2v_docs))]

12403it [00:00, 288904.92it/s]


In [37]:
d2v_model = doc2vec.Doc2Vec(size=300, window=5, alpha=0.025, min_alpha=0.025, seed=0)
d2v_model.build_vocab(tagged_d2v_docs)
d2v_model.train_lbls = False # do not train labels of words
d2v_model.train_words = True # only train relations among words

In [38]:
for epoch in range(10):
    d2v_model.train(tagged_d2v_docs, total_examples=len(d2v_docs), epochs=epoch)
    d2v_model.alpha -= 0.002 # decrease the learning rate
    d2v_model.min_alpha = d2v_model.alpha # fix the learning rate, no decay

In [39]:
X_d2v = [d2v_model.infer_vector(x.words) for x in tagged_d2v_docs]

In [40]:
cnum = 10

In [41]:
from sklearn.metrics.pairwise import cosine_distances
from sklearn.cluster import k_means_ 

def euc2cos_distance(X, Y=None, Y_norm_squared=None, squared=False):
    return cosine_distances(X,Y)

k_means_.euclidean_distances = euc2cos_distance

In [42]:
km = KMeans(n_clusters=cnum, init='k-means++', max_iter=300, n_init=20, tol=0.0001)
km.fit(X_d2v)
print("Silhouette Coefficient: %0.3f" % silhouette_score(X_d2v, km.labels_))

Silhouette Coefficient: -0.038


In [43]:
dfRes = pd.DataFrame(corpus, columns=['text'])
dfRes['label'] = km.labels_

for i in range(cnum):
    print('label{}: {}'.format(i, len(dfRes[dfRes.label == i])))

label0: 63
label1: 3644
label2: 863
label3: 529
label4: 672
label5: 245
label6: 4187
label7: 490
label8: 787
label9: 923


In [44]:
# print top keywords
def print_topwords(df, num, bar):
    try:
        temp_cor = df.text.values
        temp_vec = TfidfVectorizer(tokenizer=nav_tokenizer, stop_words=stopwords + mask1)

        temp_vec.fit(temp_cor)

        temp = pd.DataFrame(temp_vec.vocabulary_.keys(), columns=['word'])
        temp['idx'] = temp_vec.vocabulary_.values()
        temp.sort_values(by='idx', inplace=True)
        temp['count'] = temp_vec.transform(temp_cor).toarray().sum(axis=0)

        topwords = temp.sort_values(['count'], ascending=False)[:num]    
    
        for i in range(num):
            print(topwords['word'].values[i]),

        if bar == True:
            fig, ax = plt.subplots()
            plt.bar(range(num), topwords['count'].values, align='center')
            plt.xticks([x for x in range(num)], topwords['word'].values)
            ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=90)
            plt.tight_layout()
            plt.show()
        else:
            pass
    except:
        pass

In [45]:

for i in range(cnum):
    print('cluster {}:'.format(i)),
    print_topwords(df=dfRes.loc[dfRes['label'] == i], num=5, bar=False)
    print("")

cluster 0:

cluster 1:

cluster 2:

cluster 3:

cluster 4:

cluster 5:

cluster 6:

cluster 7:

cluster 8:

cluster 9:



In [46]:
cnum = 10

In [47]:
agg = AgglomerativeClustering(n_clusters=cnum, affinity='cosine', linkage='complete')
agg.fit(X_d2v)
print("Silhouette Coefficient: %0.3f" % silhouette_score(X_d2v, agg.labels_))

Silhouette Coefficient: -0.172


In [48]:
dfRes = pd.DataFrame(corpus, columns=['text'])
dfRes['label'] = agg.labels_
for i in range(len(dfRes.label.unique())):
    print('label {}: {}'.format(i, len(dfRes.loc[dfRes['label'] == i])))

label 0: 1382
label 1: 1575
label 2: 1708
label 3: 784
label 4: 3073
label 5: 631
label 6: 544
label 7: 4
label 8: 1140
label 9: 1562


In [49]:
for i in range(cnum):
    print('cluster {}:'.format(i)),
    print_topwords(df=dfRes.loc[dfRes['label'] == i], num=5, bar=False)
    print("")

cluster 0:

cluster 1:

cluster 2:

cluster 3:

cluster 4:

cluster 5:

cluster 6:

cluster 7:

cluster 8:

cluster 9:



In [54]:
dfRes = pd.DataFrame(corpus, columns=['text'])
dfRes['label'] = Y

In [55]:
for i in range(len(dfRes.label.unique())):
    if i == 0 :
        print('label -1: {}'.format(len(dfRes.loc[dfRes['label'] == i-1])))
    else:
        print('label {}: {}'.format(i-1, len(dfRes.loc[dfRes['label'] == i-1])))

label -1: 12403


In [56]:
for i in range(-1, cnum):
    print('cluster {}:'.format(i)),
    print_topwords(df=dfRes.loc[dfRes['label'] == i], num=5, bar=False)
    print("")


cluster -1:



In [57]:
cluster = hdbscan.HDBSCAN(min_cluster_size=10, prediction_data=True).fit(X_d2v)

In [58]:
cnum = len(np.unique(cluster.labels_)) - 1
cnum

0

In [59]:
lda_docs = [nav_tokenizer(x) for x in corpus]
lda_docs = [remove_stopwords(x, stopwords + mask1) for x in lda_docs]
lda_dict = corpora.Dictionary(lda_docs)
#calulate TF-IDF
tf_ps = [lda_dict.doc2bow(x) for x in lda_docs]
tf_model = models.TfidfModel(tf_ps)
X_lda = tf_model[tf_ps]

In [60]:
# train topic model
ntopics = 30
nwords = 5

np.random.seed(1)
lda_model = models.ldamodel.LdaModel(X_lda, id2word=lda_dict, num_topics=ntopics)
pprint(lda_model.print_topics(num_topics=ntopics, num_words=nwords))

[(0, '0.007*"천호진" + 0.004*"강식당" + 0.004*"신과함께" + 0.004*"신생아" + 0.003*"박종철"'),
 (1, '0.007*"고준희" + 0.005*"로사리오" + 0.005*"시상" + 0.004*"김연아" + 0.004*"시신"'),
 (2, '0.004*"수상" + 0.004*"감독" + 0.004*"이준" + 0.003*"일본" + 0.003*"역적"'),
 (3, '0.016*"연예대상" + 0.007*"애플" + 0.006*"버라이어티" + 0.006*"아이폰" + 0.005*"배터리"'),
 (4, '0.013*"계약" + 0.013*"선수" + 0.011*"두산" + 0.009*"투수" + 0.008*"구단"'),
 (5, '0.005*"이민지" + 0.005*"가구" + 0.004*"엄기준" + 0.004*"배수지" + 0.004*"가요대축제"'),
 (6, '0.007*"추자현" + 0.005*"우효광" + 0.005*"당협위원장" + 0.002*"편의점" + 0.002*"로빈"'),
 (7,
  '0.007*"비트코인" + 0.005*"가상화폐" + 0.004*"미운우리새끼" + 0.004*"박수홍" + 0.003*"거래소"'),
 (8, '0.009*"내연녀" + 0.008*"런닝맨" + 0.005*"전소민" + 0.003*"재신임" + 0.003*"이우인"'),
 (9, '0.016*"박나래" + 0.014*"전현무" + 0.008*"기안84" + 0.004*"한혜진" + 0.004*"이상윤"'),
 (10, '0.010*"강정호" + 0.007*"박시후" + 0.006*"서지안" + 0.005*"황금빛" + 0.004*"피츠버그"'),
 (11, '0.008*"선수" + 0.006*"한화" + 0.005*"양현종" + 0.005*"경기" + 0.005*"리그"'),
 (12, '0.014*"데얀" + 0.003*"김치" + 0.003*"손준호" + 0.003*"흑산" + 0.003*"아드리아노"')

In [61]:
idx = 11
pprint(corpus[idx])

('文정부 첫 감사원장 후보자에 최재형 사법연수원장(상보).\n'
 '감사원장 후보자로 지명된 최재형 사법연수원장. 2017.3.2/뉴스1 © News1 박정호 기자  (서울=뉴스1) 김현 기자 = 문재인 '
 '대통령은 7일 황찬현 전 감사원장의 퇴임으로 공석이 된 감사원장 후보자에 최재형(61) 사법연수원장을 지명했다.  최 후보자가 국회 '
 '인사청문회와 인준 표결을 거쳐 감사원장에 취임하게 되면 새 정부에서 임명된 첫 감사원장이 된다.  윤영찬 청와대 국민소통수석은 이날 '
 '춘추관에서 브리핑을 갖고 최 원장을 감사원장 후보자로 지명했다고 발표했다.  최 후보자는 경남 진해 출신으로, 경기고와 서울대 법학과를 '
 '졸업했다. 사법시험 23회(사법연수원 13기)로 서울지방법원 동부지원 판사를 시작으로 판사 생활을 걸어왔다.  대전지방법원 '
 '법원장(대전가정법원 법원장 겸임), 서울가정법원 법원장, 서울고등법원 부장판사를 거쳐 올해 2월부터 사법연수원장에 재임하고 있다.  '
 "gayunlove@news1.kr  ▶ '평창2018' 관련 뉴스·포토 보기 ▶ 네이버메인에 ‘뉴스1채널’ 설정하기! ▶ 여러분의 제보를 "
 '기다립니다.제보하기! [© 뉴스1코리아(news1.kr), 무단 전재 및 재배포 금지]')


In [62]:
lda_model[X_lda[idx]]

[(2, 0.083438501),
 (7, 0.11213528),
 (14, 0.16372836),
 (15, 0.03779294),
 (27, 0.46762866)]

In [63]:
lda_vec = np.zeros((len(corpus), ntopics))
lda_vec.shape

(12403, 30)

In [65]:
# raise small number to calculate cosine distances
lda_vec = lda_vec + 0.0001

In [66]:
X_l2v = np.hstack((X_d2v, lda_vec))

In [67]:
cnum = 10

In [68]:
km = KMeans(n_clusters=cnum, init='k-means++', max_iter=300, n_init=20, tol=0.0001)
km.fit(X_l2v)
print("Silhouette Coefficient: %0.3f" % silhouette_score(X_l2v, km.labels_))

Silhouette Coefficient: -0.032


In [69]:
dfRes = pd.DataFrame(corpus, columns=['text'])
dfRes['label'] = km.labels_

for i in range(cnum):
    print('label{}: {}'.format(i, len(dfRes[dfRes.label == i])))

label0: 584
label1: 762
label2: 8
label3: 814
label4: 525
label5: 4341
label6: 898
label7: 257
label8: 616
label9: 3598


In [70]:
for i in range(cnum):
    print('cluster {}:'.format(i)),
    print_topwords(df=dfRes.loc[dfRes['label'] == i], num=5, bar=False)
    print("")

cluster 0:

cluster 1:

cluster 2:

cluster 3:

cluster 4:

cluster 5:

cluster 6:

cluster 7:

cluster 8:

cluster 9:



In [71]:
db = DBSCAN(eps=0.30, min_samples=10, algorithm='kd_tree', metric='euclidean')
Y = db.fit_predict(lda_vec)

In [72]:
# Number of clusters in labels, ignoring noise if present.
cnum = len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)

print('Estimated number of clusters: %d' % cnum)
print("Silhouette Coefficient: %0.3f" % silhouette_score(X_l2v, db.labels_))

Estimated number of clusters: 1
Silhouette Coefficient: 0.073


In [73]:
dfRes = pd.DataFrame(corpus, columns=['text'])
dfRes['label'] = Y

In [74]:
for i in range(len(dfRes.label.unique())):
    if i == 0 :
        print('noise-1: {}'.format(len(dfRes.loc[dfRes['label'] == i-1]))),
        print(" | "),
        print_topwords(df=dfRes.loc[dfRes['label'] == i-1], num=5, bar=False)
    else:
        print('cluster {}: {}'.format(i-1, len(dfRes.loc[dfRes['label'] == i-1]))),
        print(" | "),
        print_topwords(df=dfRes.loc[dfRes['label'] == i-1], num=5, bar=False)
    print("")

noise-1: 19
 | 

cluster 0: 12384
 | 



In [75]:
cluster = hdbscan.HDBSCAN(min_cluster_size=55, prediction_data=True).fit(X_l2v)

In [76]:
cnum = len(np.unique(cluster.labels_)) - 1
cnum

0

In [77]:
# 클러스터별 Probabilities
soft_cluster = hdbscan.all_points_membership_vectors(cluster)
soft_cluster.shape

ValueError: Invalid shape in axis 1: 0.

In [ ]:
dfRes = pd.DataFrame(corpus, columns=['text'])
for i in range(cnum):
    dfRes[i] = 0; 
dfRes['noise'] = 0
dfRes['unknown'] = 0

for i in range(len(soft_cluster)):
    
    if np.isnan(soft_cluster[i][0]):
        dfRes.set_value(i, ['noise'], 1)
    elif len([x for x in soft_cluster[i] if x != 0]) == 0:
        dfRes.set_value(i, ['unknown'], 1)
    else:
        # max probability의 0.9 이상일 때 라벨링
        #idx = [j for j, x in enumerate(soft_cluster[i]) if x >= (0.9 * soft_cluster[i].max())]
        idx = [j for j, x in enumerate(soft_cluster[i]) if (x == soft_cluster[i].max()) | (x >= 0.2)]
        #idx = [j for j, x in enumerate(soft_cluster[i]) if (x == soft_cluster[i].max())]
        dfRes.set_value(i, idx, 1)

In [ ]:
print('noise-1: {} | '.format(len(dfRes[(dfRes['noise'] == 1)]))),
print_topwords(df=dfRes.loc[dfRes['noise'] == 1], num=5, bar=False)
print("")
print('unknown: {} | '.format(len(dfRes[(dfRes['unknown'] == 1)]))),
print_topwords(df=dfRes.loc[dfRes['unknown'] == 1], num=5, bar=False)
print("")


for i in range(cnum):
    print('cluster {}: {}'.format(i, len(dfRes.loc[dfRes[i] == 1]))),
    print(" | "),
    print_topwords(df=dfRes.loc[dfRes[i] == 1], num=5, bar=False)
    print("")